In [90]:
import cv2
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split


In [91]:
def preProcessing(img):
    # Check if the image is already grayscale
    if len(img.shape) == 2:
        gray = img
    else:
        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blurring with a kernel size of 5x5 and sigma of 0
    gray = cv2.GaussianBlur(gray, (5, 5), 0)


    # Apply histogram equalization to enhance the contrast of the image
    hist = cv2.equalizeHist(gray)

    return hist


In [92]:
def segmentHandRegion(gray):
    # Segment the hand region
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    contours, hierarchy = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    hand_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(hand_contour)
    hand_roi = gray[y:y+h, x:x+w]
    return hand_roi

In [93]:
def resizeImage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.resize(gray , (128 , 64))

In [94]:
def extract_hand_segment(image):
     # Convert BGR image to YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split the channels
    y_channel, cr_channel, cb_channel = cv2.split(ycrcb_image)

    # Thresholding to create a binary mask for hand region
    _, cr_mask = cv2.threshold(cr_channel, 133, 173, cv2.THRESH_BINARY)
    _, cb_mask = cv2.threshold(cb_channel, 77, 127, cv2.THRESH_BINARY)

    # Combine the masks
    mask = cv2.bitwise_and(cr_mask, cb_mask)
    # Find the contours of the object in the mask
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (which should be the hand)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]

    # Create a mask of the hand contour
    hand_mask = np.zeros_like(mask)
    cv2.drawContours(hand_mask, [largest_contour], 0, 255, cv2.FILLED)

    # Dilate the hand mask to fill in any gaps
    kernel = np.ones((1, 1), np.uint8)
    dilated_mask = cv2.dilate(hand_mask, kernel, iterations=1)

    # Apply erosion to reduce the size of the segmented area
    kernel = np.ones((1, 1), np.uint8)
    eroded_mask = cv2.erode(dilated_mask, kernel, iterations=1)

    # Apply the hand mask to the original image to extract the hand
    hand_segment = cv2.bitwise_and(image, image, mask=eroded_mask)

    # Return the segmented hand image
    return hand_segment


In [95]:
# Define the image transformation functions
def flip(image):
    # Flip the image horizontally
    return cv2.flip(image, 1)

def rotate(image):
    # Rotate the image by 30 degrees
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 30, 1)
    return cv2.warpAffine(image, M, (cols, rows))

In [96]:
count = 0
# Prepare the training data
X = []
y = []
images = []
for i in range(0, 6):
    for j in range(1, 180):
        filename = 'men/{}/{}_men ({}).JPG'.format(i, i , j)
        img = cv2.imread(filename)
        if  img is None:
            continue    
        
        hand_roi = extract_hand_segment(img)
        finalImg = resizeImage(hand_roi)
        images.append(finalImg)
        y.append(i)
        count += 1


        filename = 'Woman/{}/{}_woman ({}).JPG'.format(i, i, j)
        img = cv2.imread(filename)
        if  img is None:
            continue    
        

        hand_roi = extract_hand_segment(img)
        finalImg = resizeImage(hand_roi)
        images.append(finalImg)
        y.append(i)
        count += 1


        


In [97]:
hog_features = hog(images[0], orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')
X = np.zeros((len(y) , len(hog_features)))
for i in range(len(y)):
     hog_features = hog(images[i], orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')
     X[i] = (np.array(hog_features))
print(len(X))     
print(len(y))     

1796
1796


In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [99]:
# Train the SVM model
model = svm.SVC(kernel='linear')
model.fit(X_train, y_train)
    

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [100]:
# Evaluate the model
accuracy = model.score(X_test, y_test)


In [101]:
print(f'Accuracy: {accuracy*100}%')

Accuracy: 76.94444444444444%


In [102]:
print(count)

1796
